# Hearthstone Project
***
***

# Goals
- Explore the data to gather insights about the characteristics of the game's different class types

# Purpose
- Practice visualizing data using a variety of different plot types and configurations

# Setup
***

In [ ]:
# establishing environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from acquire import get_hearth
from prepare import prep_hearth

import warnings
warnings.filterwarnings("ignore")

# Acquire
Acquiring data from local csv files
***

In [ ]:
# reading in card data and saving as DF
cards = pd.read_csv('hearthstone_standard_cards.csv')

# reading in card classes and saving as DF
classes = pd.read_csv('classes.csv')

# reading in minion types data and saving as DF
mtypes = pd.read_csv('minionTypes.csv')

# reading in types data and saving as DF
ctypes = pd.read_csv('types.csv')

# reading in keywords data and saving as DF
keywords = pd.read_csv('keywords.csv')

- Data acquired from local csv files
- Alternatively, the __get_hearth__ function from the __acquire.py__ file can be run to acquire the data
    - The DFs will be returned in the following order by the function: cards, classes, mtypes, ctypes, keywords

# Prepare
Preparing data for exploration


__IMPORTANT NOTE: Some preparations to the data will be done solely for the purpose of practicing preparation techniques and will not be relevant to the exploration phase.__
***

## Preparing DataFrames for Merge

### Lowercasing column names and values
### Dropping unneeded columns and changing various column names

In [ ]:
# lowercasing cards DF columns
cards.columns = cards.columns.str.lower()

# lowercasing name and text column values
cards.text = cards.text.str.lower()
cards.name = cards.name.str.lower()

# creating list of all DFs besides cards
df_list = [classes, mtypes, ctypes, keywords]

# iterating through DFs
# lowercasing all column names, dropping original name column, renaming slug to name column
for dtafrm in df_list:
        dtafrm.columns = dtafrm.columns.str.lower()
        dtafrm.drop(columns = 'name', inplace = True)
        dtafrm.rename(columns = {"slug": "name"}, inplace = True)       

## Merging DataFrames

### Merging 'classes' DF

In [ ]:
# removing brackets and commas from multiclassids column
cards.multiclassids = cards.multiclassids.str.replace('\]|,|\[' , '')

# creating column to hold primary class id 
# if card is of one class, this will reflect its sole class
# if card is dual, this will reflect the 1st of the two classes in the multiClassIds column
# necessary since dual class cards erroneously hold the 'neutral' class value in their primary class id 
cards['primeclassid'] = np.where((cards.multiclassids.str.contains(' ')), cards["multiclassids"].str.split(" ", expand = True)[0], cards.classid)

# converting key columns to make all value data types match
cards.primeclassid = cards.primeclassid.astype(str)
classes.id = classes.id.astype(str)

# merging 'classes' df with card df
df = pd.merge(cards, classes[['id', 'name']], 
              left_on = 'primeclassid', right_on = 'id', how="left", 
              suffixes = (None, '_prime_hero_class'))

# dropping columns I no longer need
df.drop(columns = ['primeclassid', 'classid', 'duels'], inplace = True)

### Merging 'mtypes' DF

In [ ]:
# changing null values of minionTypeId for neutral minions to -1
df['miniontypeid'] = np.where((df.miniontypeid.isnull() == True) & (df.cardtypeid == 4), -1, df.miniontypeid)

# adding missing keyword data to 'keywords' df
# -1 is for minions with no tribe
mtypes.loc[len(mtypes.index)] = ['no_tribe', -1]

# merging 'mtypes' df
df = pd.merge(df, mtypes[['id', 'name']], 
              left_on = 'miniontypeid', right_on = 'id', how="left", 
              suffixes = (None, '_minion_type'))

# dropping column I no longer need
df.drop(columns = ['miniontypeid'], inplace = True)

### Merging 'ctypes' DF

In [ ]:
# merging 'ctypes' df
df = pd.merge(df, ctypes[['id', 'name']], 
              left_on = 'cardtypeid', right_on = 'id', how="left", 
              suffixes = (None, '_card_type'))

# dropping column I no longer need
df.drop(columns = ['cardtypeid'], inplace = True)

### Merging 'keywords' DF

In [ ]:
# replacing dashes with underscores in names
keywords.name = keywords.name.str.replace('-', '_')

# adding missing keyword data to 'keywords' df
keywords.loc[len(keywords.index)] = ['64', 'start_of_game', 
                                     'does something at the start of the game.', 
                                     'does something at the start of the game.']

# removing brackets and commas from keyword id column
df.keywordids = df.keywordids.str.replace('\]|,|\[' , '')

# splitting keyword ids into separate columns for each card
kwdf = df["keywordids"].str.split(" ", expand = True) 

# renaming columns
kwdf.columns = ['keywordid1', 'keywordid2', 'keywordid3', 'keywordid4', 'keywordid5']

# concatenating split keyword id columns with main df
df = pd.concat([df, kwdf], axis=1)

# converting keywords id column to str type to enable merge
keywords.id = keywords.id.astype(str)

# creating loop to add a column for the text name of each keyword ability of each card
# via merging with keywords DF
for x in kwdf.columns:
    df = pd.merge(df, keywords[['id', 'name']], 
              left_on = x, right_on = 'id', how = "left",
              suffixes = (None, x + '_name'))

## Checking for duplicate rows

In [ ]:
# checking number of rows in current DF
df.shape

In [ ]:
# Checking number of rows if duplicates were dropped
df.drop_duplicates().shape

- No duplicates found

### Checking for proper data types, categorical columns (based on domain knowledge), and null counts

In [ ]:
df.info()

- The following columns will be dropped as they won't be needed for the expected operations of this project
    - id, slug
        - unique identifiers for cards, not needed since the 'name' column provides this while also being easier to reference
    - artistname, cardsetid, duels
        -  I don't anticipate that the names of the cards' artists, their set names, or their existence in the duels format will confer any insight about the characteristics of their respective hero classes
    - image, imagegold, cropimage
        - I won't be exploring the images of the cards in this iteration of the project but I may in the future
    - durability, armor
        - Durability is exclusive to weapons and armor is exclusive to hero cards
        - Weapons and Hero cards represent a small minority of cards compared to spells and minions, and some classes have no weapons and/or hero cards currently
        - In light of these observations, I don't believe I'll be able to make any solid inferences about class identities with either of these columns
    - collectible
        - Only 1 value, no nulls, doesn't distinguish any cards
    - all columns reflecting key words with the exception of the boolean columns and the 'slug_keyword#_name' columns
        - The exempted columns are sufficient for the project's expected operations
     
     
- Based on my domain knowledge of the game, I'm inferring that several of the columns are categorical
    - I need to create boolean columns for categorical columns (rarity, card set, etc.)


- Many null values that need to be addressed
    - text
    - duels
    - minion type id
    - health
    - attack
    - child ids
    - durability
    - armor

### Dropping columns that aren't needed for the planned operations of this project

In [ ]:
# creating list of columns to drop
columns_to_drop = ['id', 'slug', 'artistname', 'image', 'imagegold', 'flavortext', 'cropimage', 'collectible', 'durability', 'armor']

# dropping columns
df.drop(columns = columns_to_drop, inplace = True)

## Addressing Null Values

### Addressing nulls in 'text' column

In [ ]:
# checking values in text box
df.text.value_counts(dropna = False)

In [ ]:
# filling null text values with 'no effect'
df["text"].fillna("no effect", inplace = True) 

### Addressing nulls in 'id_minion_type', and 'slug_minion_type' columns

In [ ]:
# checking minontypeId values
df.id_minion_type.value_counts(dropna = False)

In [ ]:
# converting nulls, aka non-minion cards to 'not a minion' type
df['id_minion_tribe'] = np.where((df.id_minion_type.isnull() == True), 'not a minion', df.id_minion_type)
df['name_minion_tribe'] = np.where((df.name_minion_type.isnull() == True), 'not a minion', df.name_minion_type)

# dropping minionTypeId since id_minion_type suffices
df.drop(columns = ['id_minion_type', 'name_minion_type'], inplace = True)

### Addressing nulls in 'childIds' column

In [ ]:
# checking childIds values
df.childids.value_counts(dropna = False)

In [ ]:
# filling nulls with "no_childid"
df.childids.fillna("no_childid", inplace = True) 

### Addressing nulls in 'health' and 'attack' columns
These variable's respective columns have null values since neither of them apply to every card (examples: minions have health but spells and weapons do not). For the time being I'll fill these nulls 
with a value that represents infinity since doing so allows me to preserve the int64 data type of the column. Furthermore, no matter what value the developer assigns to these variables in future cards, this value probably won't be used. If this causes issues later I'll employ a different means of handling them. 

In [ ]:
# creating list of column names
hlth_atk = ['health', 'attack']

# iterating through columns filling nulls within each
for att in hlth_atk:
    df[att].fillna(float('inf'), inplace = True)

## Creating boolean columns for categorical variables

### Creating boolean columns for 'keywords'

In [ ]:
# loop iterates through each keyword and creates a boolean column for it
for kw in keywords.name:
    df['has_' + kw] = np.where(
    (df.namekeywordid1_name == kw) |
    (df.namekeywordid2_name == kw) |
    (df.namekeywordid3_name == kw) |
    (df.namekeywordid4_name == kw) |
    (df.namekeywordid5_name == kw), 1, 0)
    
# creating empty list
key_word_col_drop = []

# iterating through columns in df and creating list of columns to drop
for col in df.columns:
    if 'keywordid' in col:
        key_word_col_drop.append(col)
        
# dropping columns
df.drop(columns = key_word_col_drop, inplace = True)

### Creating boolean columns for 'hero classes'

In [ ]:
# removing brackets and commas from multiclassids column
df.multiclassids = df.multiclassids.str.replace('\]|,|\[' , '')

# creating column that holds secondary class separate from primary class
df['id_second_hero_class'] = df["multiclassids"].str.split(" ", expand = True)[1]

# converting column to str type to enable merge with newly created column 'id_second_hero_class'
classes.id = classes.id.astype(str)

# creating df containing columns for merge in order to rename before merge without altering original classes DF
classes2 = classes[['id', 'name']]

# renaming columns
classes2.columns = ['id_second_hero_class', 'name_second_hero_class']

# merging 'classes' on secondary hero class id to get secondary class names
df = pd.merge(df, classes2[['id_second_hero_class', 'name_second_hero_class']], 
              on = 'id_second_hero_class', how = "left")

# creating boolean columns for each hero class
for c in classes.name:
    df['is_' + c] = np.where(
    (df.name_prime_hero_class == c) | (df.name_second_hero_class == c), 1, 0)

# filling nulls in new columns
df['name_second_hero_class'].fillna('monoclass', inplace = True)
df['id_second_hero_class'].fillna('monoclass', inplace = True)

### Creating boolean column for multiclass cards

In [ ]:
# creating column where 1 = multiclass, 0 = monoclass)
# contains ' ' will suffice since only cards with a space in this value are multiclass
df['is_multiclass'] = np.where((df.multiclassids.str.contains(' ')), 1, 0)

# dropping column I no longer need
df.drop(columns = 'multiclassids', inplace = True)

### Creating boolean column for cards with child ids

In [ ]:
# creating column where 1 = card has childids, 0 = card has no childids)
# contains ',' will suffice since only cards with a comma in this value have childids
df['has_child_ids'] = np.where((df.childids.str.contains(',')), 1, 0)

# dropping column I no longer need
df.drop(columns = 'childids', inplace = True)

### Creating boolean columns for card type

In [ ]:
# iterating through card types and creating a boolean column for each
for ctype in ctypes.name:
    df['is_' + ctype] = np.where((df.name_card_type == ctype), 1, 0)

# dropping column I no longer need
df.drop(columns = 'id_card_type', inplace = True)

### Creating boolean columns for minion tribe

In [ ]:
# iterating through minion tribes and creating a boolean column for each
for mtype in mtypes.name:
    df['is_' + mtype] = np.where((df.name_minion_tribe == mtype), 1, 0)

# dropping column I no longer need
df.drop(columns = 'id_minion_tribe', inplace = True)

## Other Preparations

### Dropping boolean columns with all 0 values
I know that some of the keywords that were turned into booleans are not currently in standard (the format the collection of the cards in the data are part of). These columns will be completely filled with 0s so I'm going to drop them.

In [ ]:
# creating list of index values for columns that only have 0 values
all_0_cols = np.where(df.isin([0]).all() == True)

# dropping columns based on index value
df.drop(df.columns[all_0_cols], axis = 1, inplace = True)

### Adding rows for dual class cards with prime and secondary class swapped
Adding these rows will make it easier to perform certain operations such as grouping and plotting.

In [ ]:
# making identical dfs of all dual class cards
dcc = df[df.name_second_hero_class != 'monoclass']
dcc2 = df[df.name_second_hero_class != 'monoclass']

# swapping primary and secondary hero class values
dcc2.name_prime_hero_class, dcc2.name_second_hero_class, dcc2.id_prime_hero_class, dcc2.id_second_hero_class = dcc.name_second_hero_class, dcc.name_prime_hero_class, dcc.id_second_hero_class, dcc.id_prime_hero_class

# adding new rows to main df
df = pd.concat([df, dcc2])

# resetting index
df.reset_index(drop = True, inplace = True)

### Adding column that holds count of words in card name

In [ ]:
# counting words in card names and adding as variable
df['name_word_count'] = df.name.apply(lambda x: len(str(x).split(' ')))

### Adjusting column order

In [ ]:
# adjusting order of columns
df = df[['manacost', 'name', 'name_word_count', 'text', 'has_child_ids', 'health', 'attack',
       'id_prime_hero_class', 'name_prime_hero_class', 
       'id_second_hero_class', 'name_second_hero_class', 'name_card_type',
       'name_minion_tribe', 'has_taunt', 'has_spellpower', 'has_divine_shield',
       'has_charge', 'has_secret', 'has_stealth', 'has_battlecry',
       'has_freeze', 'has_windfury', 'has_deathrattle', 'has_combo',
       'has_overload', 'has_silence', 'has_counter', 'has_immune',
       'has_discover', 'has_quest', 'has_poisonous', 'has_lifesteal',
       'has_rush', 'has_evilzug', 'has_twinspell', 'has_mega_windfury',
       'has_reborn', 'has_empower', 'has_outcast', 'has_spellburst',
       'has_sidequest', 'has_corrupt', 'has_start_of_game',
       'is_demonhunter', 'is_druid', 'is_hunter', 'is_mage', 'is_paladin', 'is_priest',
       'is_rogue', 'is_shaman', 'is_warlock', 'is_warrior', 'is_neutral',
       'is_multiclass','is_hero', 'is_minion', 'is_spell',
       'is_weapon', 'is_murloc', 'is_demon', 'is_mech', 'is_elemental',
       'is_beast', 'is_totem', 'is_pirate', 'is_dragon', 'is_all',
       'is_no_tribe']]

# Preparation Phase Summary
- Lowercased all column names and values

- Merged all dataFrames into single dataframe

- Added missing keyword value (start of game)

- Added rows for dual class cards with primary and secondary hero class values swapped

- Changed all dashes in column names to underscores

- Checked for duplicate rows, none found

- Filled nulls with different values based on column and intended operations

- Created boolean columns for categorical values

- Dropped columns that were unuseful for the operations of this project

- Reordered columns

- Alternatively, the __prep_hearth__ function from the __prepare.py__ file can be run (with the 5 separate DFs made from the local csv files as arguments) to prep the data.


# Explore
Exploring the data to identify the characteristics of each class, including neutral

### Creating dictionary of colors, list of colors, and color mapper function for plots

In [ ]:
# creating dictionary that holds each class as a key with a color
# can be used to specify the color in plots where a dictionary is accepting for mapping colors to values
colord = {'Rogue' : 'gray', 'Shaman' : 'aqua', 'Hunter' : 'lime', 'Warrior' : 'red', 'Priest' : 'white', 
          'Paladin' : 'gold', 'Mage' : 'cornflowerblue', 'Warlock' : 'mediumorchid', 'Demonhunter' : 'green',
          'Druid' : 'saddlebrown', 'Neutral' : 'pink'}

# sorting colorsby corresponding class hero in alphabetical order
sorted_color_df = pd.DataFrame.from_dict(colord, orient='index').sort_index()
# creating list of colors in DF above
sorted_colors = [i for x in sorted_color_df.values for i in x]

# function accepts list of of hero classes and returns sorted list of colors for plotting
# useful when one of the items above can't be applied (ie. customer sorting of hero classes)
def color_mapper(hero_class_list):
    color_list = []
    for x in hero_class_list:
        color_list.append(colord[x])
    return color_list

### Setting plot figure sizes when size is not specified otherwise

In [ ]:
plt.rcParams["figure.figsize"] = (12,8)

### Creating separate DFs for each major card type

In [ ]:
minions = df[df.is_minion == 1]
heros = df[df.is_hero == 1]
spells = df[df.is_spell == 1]
weapons = df[df.is_weapon== 1]

### Creating list of class names

In [ ]:
classnames = classes.name.to_list()

## Percentage of Minions and Spells within each Class
Since weapons and hero cards make up only a small minority (if any) of the cards within each class, I'm not going to include them in the visualization below.
#### NOTE: I'm aware pie charts are commonly considered bad practice. Since this is a personal projet, I'm making this simply for exhibition purposes.

In [ ]:
# creating df with all cards except hero cards and weapons
donutplotdf = df[(df.name_card_type != 'hero') & ((df.name_card_type != 'weapon'))]

# creating df of card counts for each hero class
hero_dist_df = pd.DataFrame(donutplotdf.name_prime_hero_class.value_counts().sort_index())
hero_dist_df.columns = ['hero_class_card_count']

# creating df of card counts per card type per hero class
card_type_dist_df = pd.DataFrame(donutplotdf.groupby(['name_prime_hero_class', 'name_card_type']).count()['name'].sort_index(level=['name_prime_hero_class']))

# setting values for outer ring
major_pie_groups = [classname.title() for classname in hero_dist_df.index]
major_group_sizes = [percent for percent in hero_dist_df.hero_class_card_count]

# setting values for inner ring
minor_pie_groups = [x[1] for x in card_type_dist_df.to_dict()['name'].keys()]
minor_group_sizes = [cardtypecount for numlist in card_type_dist_df.values.tolist() for cardtypecount in numlist]

# creating list for minion and spell colors
min_and_spell_colors = [word.replace('minion' ,'orange') for word in minor_pie_groups]
min_and_spell_colors = [word.replace('spell','lightblue') for word in min_and_spell_colors]

# creating outside ring
fig, ax = plt.subplots(figsize=(12,12))
ax.axis('equal')
mypie, _, autotexts = ax.pie(major_group_sizes, radius=1.2, labels = major_pie_groups, autopct='%1.1f%%', pctdistance= .83, colors = sorted_colors, textprops={'fontsize': 22})
plt.setp(mypie, width=0.4, edgecolor='black')

# creating inner ring
mypie2, _, autotexts2 = ax.pie(minor_group_sizes, radius=1.2-0.4, autopct='%1.1f%%', pctdistance= .84, colors = min_and_spell_colors, textprops={'fontsize': 15})
plt.setp( mypie2, width=0.4, edgecolor='black')
plt.margins(0,0)

# creating color list for inner circle
minion_legend_item = mpatches.Patch(color='orange', label='Minions')
spell_legend_item  = mpatches.Patch(color='lightblue', label='Spells')

# creating custom legend
leg = plt.legend(handles=[minion_legend_item, spell_legend_item], prop={"size":20}, bbox_to_anchor=(0.06, .95))

# adding title to plot
plt.title('Minion and Spell Percentages Per Hero Class', fontsize = 25, pad = 45)

# displaying plot 
plt.show()

- The purpose of this visualization was to give an idea of about how the distribution of card types within each class (for example, do mages have a lot more spells than minions? do hunters have mostly minions?, etc)
    - Since I alread know that weapons and hero cards make up only a small minority of all cards, I removed them from the data used in the visualization above since their "slices" within the plot would be too small to offer meaningful insight beyond that they are the minority card types


- Neutral minions account for nearly 1/3 of all cards


- There are no neutral spell cards


- The combined amount of spells and minions within each non-neutral class are relatively close
    - Druid, Priest, and Warlock are tied for the majority at 7.5%
    - Demon Hunter has the lowest amount, making up 5.9% of all minions and spells
    
    
- All non-neutral hero classes have relatively even splits in their ratios of spells to minions

## Average Mana Costs Per Class

In [ ]:
# creating df of average mana costs per class
avgmana = pd.DataFrame(df.groupby(['name_prime_hero_class']).mean().sort_values(by = 'manacost', ascending = True)['manacost'])
avgmana.index = avgmana.index.str.title()
avgmana

In [ ]:
# creating plot
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 14)
plt.title("Average Mana Costs By Class", fontsize = 20)
ax = sns.barplot(x = "manacost", y = avgmana.index, data = avgmana, palette = colord, edgecolor = 'black')
ax.set_xlabel('Average Mana Cost', fontsize = 15)
ax.set_ylabel('Hero Class', fontsize = 17)
plt.show()

- Average mana costs range from ~3 to ~4 mana
    - Although this range may seem small, 1 mana is considered a significant difference in the viability of a card
- Rogue does has the lowest average mana cost, ~3
- Neutral has the highest average mana cost, ~4
- Of all non-neutral classes, Druid has the highest average mana cost, ~3.9

## Average Mana Costs for Each Class (Per Card Type) 

In [ ]:
pd.DataFrame(df.groupby(['name_prime_hero_class', 'name_card_type']).mean()['manacost'])

In [ ]:
# creating empty dictionaries
minion_mc_dict = {}
weapon_mc_dict = {}
spell_mc_dict = {}
hero_mc_dict = {}

# iterating through class names and storing average manacost for each type of card within each class
for classname in classnames:
    minion_mc = minions[minions.name_prime_hero_class == classname].manacost.mean()
    weapon_mc = weapons[weapons.name_prime_hero_class == classname].manacost.mean()
    spell_mc = spells[spells.name_prime_hero_class == classname].manacost.mean()
    hero_mc = heros[heros.name_prime_hero_class == classname].manacost.mean()
    
    # adding values to dictionary for use in plot
    minion_mc_dict[classname] = minion_mc
    weapon_mc_dict[classname] = weapon_mc
    spell_mc_dict[classname] = spell_mc
    hero_mc_dict[classname] = hero_mc
    
x = np.arange(len(classnames))
width = 0.2 

# creating plots
fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(x - width * 1.5, minion_mc_dict.values(), width, label= 'Minions', color = 'orange')
ax.bar(x - width * .5, weapon_mc_dict.values(), width, label = 'Weapons', color = 'crimson')
ax.bar(x + width * .5, spell_mc_dict.values(), width, label= 'Spells', color = 'lightblue')
ax.bar(x + width * 1.5, hero_mc_dict.values(), width, label = 'Heros', color = 'mediumorchid')

# setting x labels, y labels, xticks, etc.
ax.set_xlabel('Hero Class', fontsize = 20)
ax.set_ylabel('Mana Cost', fontsize = 20)
ax.set_title('Average Mana Cost by Class (Per Card Type)', fontsize = 21)
ax.set_xticks(x)
ax.set_xticklabels([classname.title() for classname in classnames], fontsize = 15, rotation = 10)
ax.legend(bbox_to_anchor=(1, 1), fontsize='x-large')
plt.show()

__General__
- Any bars that are missing from a class means that the class does not have any of that particular type of card
    - For example, mages don't have weapons currently
   
   
__Minions__    
- Druids have the more costly creatures
    - It makes sense that they have more expensive creatures since part of their identity is ramping up resources to play big creatures (and spells)
- Rogues have the cheapest creatures
    - This also makes sense given that part of their identity is being a fast-paced class that plays lots of inexpensive cards


__Weapons__
- The average cost of weapons for each class that has them is very similar with the exception of neutral which has much cheaper weapons


__Spells__
- The range of costs among spells is ~4 - ~5 mana
    - Rogues, have the cheapest spells
        - This is not surprising since again, they are known for have lots of small effect, cheap cards
    - Mages have the most expensive spells
        - This makes sense given that as the name would imply, mages focus on magic and part of their identity is having big, impactful spells, even more so than druid    
        
        
__Heros__
- No hero class has more than 1 hero card and some have non
- Furthermore, every hero class except for Mage share the same hero card, Galakrond, which explains why several have the same mana cost
       

## Average Word Counts in Name per Class

In [ ]:
# creating df of average number of words in name per class 
avgnamewc = pd.DataFrame(df.groupby(['name_prime_hero_class']).mean().sort_values(by = 'name_word_count', ascending = True)['name_word_count'])
avgnamewc.index = avgnamewc.index.str.title()
avgnamewc

In [ ]:
# creating plot
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 14)
plt.title("Average Word Count in Name Per Class", fontsize = 20)
ax = sns.barplot(x = "name_word_count", y = avgnamewc.index, data = avgnamewc, palette = colord, edgecolor = 'black')
ax.set_xlabel('Average Word Count in Name', fontsize = 15)
ax.set_ylabel('Hero Class', fontsize = 17)
plt.show()

- Very little variation in the word counts in the name's of each classes' cards
    - Range is ~1.8 to ~2.3 words
- Rogues have the lowest average words counts
- Paladins have the highest average word counts

## Average Name Word Count by Class (per Card Type)

In [ ]:
df.groupby(['name_prime_hero_class', 'name_card_type']).mean()['name_word_count']

In [ ]:
# creating dictionary for colors for each card type
ctype_colord = {'Spell' : 'lightblue', 'Minion' : 'orange', 'Hero' : 'mediumorchid', 'Weapon' : 'crimson'}

# copying main df to avoid altering
wc_by_ctype_df = df.copy()

# capitalizing hero class names and card types for aesthetic appeal in plot
wc_by_ctype_df.name_prime_hero_class = wc_by_ctype_df.name_prime_hero_class.str.title()
wc_by_ctype_df.name_card_type = wc_by_ctype_df.name_card_type.str.title()

# creating plot
plt.figure(figsize=(15,8))
plt.xticks(fontsize = 15, rotation = 10)
ax = sns.barplot(x = "name_prime_hero_class",  y = "name_word_count", hue = 'name_card_type',  data = wc_by_ctype_df, ci = None, palette =  ctype_colord)
plt.xlabel('Hero Class', fontsize = 20)
plt.ylabel('Average Name Word Count', fontsize = 20)
plt.title('Average Name Word Count by Class (per Card Type)', fontsize = 21)
ax.legend(bbox_to_anchor=(1, 1), fontsize='x-large')
plt.show()


__General__
- Any bars that are missing from a class means that the class does not have any of that particular type of card
    - For example, druid don't have weapons or hero cards currently
- With the exception of hero cards (covered below), paladin weapons, and neutral weapons, all of the groups of fell into the range of 1.5 to 2.5 average words per name
    - This implies that there is little variance (roughly 1 word) between the average number of words for the cards of each group

- Of all non-neutral classes, paladins have the most words on average for both weapons and spells in addition to nearly being tied for minions
   

__Spells__
- Paladins have the highest word counts in their spell names on average
- Rogues have the lowest word counts in their spell names on average
        
        
__Minions__   
- Priests narrowly beat out paladins for the highest average word count in minion names
- Shamans closely edged out hunters for the lowest average word count in minion names


__Weapons__
- The neutral "class" has the largest highest average word count in weapon names, albeit there is only one neutral weapon currently (Sphere of Sapience)
- With regard to non-neutral classes, paladins have the highest average word count in weapon names
- Demon hunter has the lowest average word count in weapon names

        
__Heros__
- No hero class has more than 1 hero card and some have non
- Furthermore, every hero class except for Mage share the same hero card, Galakrond, which explains why several have the same word count
       

## Variables: Health and Attack
I'll be exploring these variables in tandem since they are closely related. Health determines how much damage a minion can take before being eliminated while attack determines how much damage a minion can output.

In [ ]:
# creating series of average minion health, sorted alphabetically by hero class name
minions_health = minions.groupby('name_prime_hero_class').mean()['health'].sort_values()
minions_health.index = minions_health.index.str.title()

# creating list of colors for hero class colors based on order in minions_health series
health_colors = color_mapper(minions_health.index)

# plotting horizontal lines (stems of lollipops)
ytick_range = range(0,len(minions_health.index))
plt.hlines(y = ytick_range, xmin = 0, xmax = minions_health - .06, color = 'b')

# plotting dots ("candy" portion of lollipop) that reflect average minion health for respective class
plt.scatter(minions_health, ytick_range, color = health_colors , s = 300, edgecolors= 'b')

# configuring ticks, axis labels, etc.
plt.yticks(ytick_range, minions_health.index, fontsize = 15)
plt.xticks(fontsize = 15)
plt.ylabel('Hero Class', fontsize = 20)
plt.xlabel('Minion Health', fontsize = 20)
plt.title('Average Minion Health by Class', fontsize = 25)

plt.show()

- The range of the average health between minions of different classes is relatively substantial
    - A minion losing or gaining 1 health as a result of a balancing patch usually has a major impact on their viability


- Rogue minons have the lowest average health among all minions
    - This is consistent with their propensity for being a class that plays cheaper, weaker cards in large volumes to compensate for their lack of individual impact
    
    
- Druid minions have the highest average health among all minions
    - This makes sense given that one of their main strategies is to ramp up resources in order to play more powerful spells and minions so the developers gave them "bulkier" (ie. higher health) creatures on average. 
        - Bulkier creatures are harder to remove, so druid is rewarded for their investment, in part, by having minions that are harder to destroy

In [ ]:
# creating series of average minion attack, sorted alphabetically by hero class name
minions_attack = minions.groupby('name_prime_hero_class').mean()['attack'].sort_values()
minions_attack.index = minions_attack.index.str.title()

# creating list of colors for hero class colors based on order in minions_attack series
attack_colors = color_mapper(minions_attack.index)

# plotting horizontal lines (stems of lollipops)
ytick_range = range(0,len(minions_attack.index))
plt.hlines(y = ytick_range, xmin = 0, xmax = minions_attack - .06, color = 'crimson')

# plotting dots ("candy" portion of lollipop) that reflect average minion attack for respective class
plt.scatter(minions_attack, ytick_range, color = attack_colors, s = 300, edgecolors= 'crimson')

# configuring ticks, axis labels, etc.
plt.yticks(ytick_range, minions_attack.index, fontsize = 15)
plt.xticks(fontsize = 15)
plt.ylabel('Hero Class', fontsize = 20)
plt.xlabel('Minion attack', fontsize = 20)
plt.title('Average Minion Attack by Class', fontsize = 25)

plt.show()

- The range of average minion attack values between classes is ~3 - ~4
    - Although the range isn't as large as minion health, it is a significant range nonetheless. Like with minion health, 1 increase or decrease to a minions attack can have a significant impact on its competitive viability
    
    
- The order of the classes has changed from the order in the health plot


- Demon hunter minions have the highest average attack
    - Demon hunters are known to be an agressive class, as opposed to classes like priest which are more defensive by nature.


- Mage minons are nearly tied with shamans for the lowest average attack values
    - I'm surpised by this, I figured that this rank would have been held by one of the more defensive classes such as priest 


In [ ]:
# creating df that holds both average minion attack and health
atk_and_hlth_df = pd.DataFrame(dict(Average_Health = minions_health, Average_Attack = minions_attack))
atk_and_hlth_df.columns = ['Average Health', 'Average Attack']

# creating list of colors for horizontal lines
hlth_attk_colors = color_mapper(atk_and_hlth_df.index)

# setting y tick range
ytick_range = range(0,len(atk_and_hlth_df.index))

x_min_val = np.where(atk_and_hlth_df.index != 'Demonhunter', atk_and_hlth_df['Average Health'] * .99, atk_and_hlth_df['Average Health'] * 1.012)
x_max_val = np.where(atk_and_hlth_df.index != 'Demonhunter', atk_and_hlth_df['Average Attack'] * 1.012, atk_and_hlth_df['Average Attack'] * .99)

# creating scatter plot points for attack and health
plt.scatter(atk_and_hlth_df['Average Health'], ytick_range, color='b', alpha=1, label='Average Health', s = 200)
plt.scatter(atk_and_hlth_df['Average Attack'], ytick_range, color='crimson', label='Average Attack', s = 200)

# creating horizontal lines to connect attack and health along with black outline
# have to create larger black bar in background since hlines edgecolor argument fills entire line instead of outline
ax = plt.hlines(y = ytick_range, xmin = x_min_val, xmax = x_max_val, colors = 'black', linewidth= 7)
ax = plt.hlines(y = ytick_range, xmin = x_min_val, xmax = x_max_val , colors = hlth_attk_colors, linewidth= 5)
 
# setting xticks, yticks, axis labels, title, and legend
plt.xticks(range(0,6,1), fontsize = 15)
plt.yticks(ytick_range, atk_and_hlth_df.index, fontsize = 15)
plt.title('Average Attack vs Average Health by Class', fontsize = 25)
plt.xlabel('Average Attack / Health', fontsize = 20, labelpad = 10)
plt.ylabel('Hero Class', fontsize = 25, labelpad = -20)
plt.legend(bbox_to_anchor=(1, 1), fontsize='x-large')

plt.show()

- The range of average attack and health varies significantly between some classes


- Warrior, Warlock, Shaman, Priest and Hunter have approximately 1 more health than attack on average


- Paladin, Neutral, and Hunter have approximately .5 more health than attack on average


- Rogue and Demonhunter have almost identical attack and health on average


- Uniquely, demonhunter is the only class with minions that have higher attack than health on average

In [ ]:
df.columns